# Formatear Datos y convertirlos en un Dataframe

Cuando descargamos el chat de un grupo de Whatsapp viene en el siguiente formato:

![Alt text](../data/chat_txt.png)

Cuando tenemos registrado el miembro en el teléfono, nos aparece en el apartado de usuario el nombre de registro. Cuando no lo tenemos registrado, nos muestra el número de teléfono.
Vamos a crear algunas funciones para obtener la información de interés en un formato adecuado para luego convertirlo en un DataFrame.  

## Librerías

In [1]:
import re
import pandas as pd

## Función para extraer Fecha y Hora

In [2]:
def Date_Chat(l):
    pattern = '^[0-9][0-9]?(\/)[0-9][0-2]?(\/)[0-9][0-9]\s[0-9][0-9]?:[0-9][0-9]\s[p|a]\.\s[m]\.\s-'
    result = re.match(pattern, l)
    
    # retorna True si encuentra una fecha y hora conjuntas
    if result:
        return True
    return False

## Función para extraer el Usuario

In [3]:
def IsAuthor(l):
    pattern = '([+]\d{2} \d{3} \d{7}):'

    patterns = '^' + '|'.join(pattern)
    result = re.match(patterns, l)

    # retorna True si encuentra un numero telefonico o nombre de usuario
    if result:
        return True
    return False

## Función para extraer las coincidencias (Match) y convertirlas en un DataPoint

In [4]:
def DataPoint(line):
    SplitLine = line.split(' - ')       # divide la linea en el signo -

    DT = SplitLine[0]                   # almacena la primera parte de la linea en DT

    DateTime = DT.split(' ')            # divide la linea donde haya espacios 
    Date = DateTime[0]                  # almacena la fecha en Date
    Time = DateTime[1]                  # almacena la hora en Time
    Format = DateTime[2]                # almacena el formato de la hora en Format

    Message = ' '.join(SplitLine[1:])   # almacena la segunda parte de la linea en Message
    
    if IsAuthor(Message):
        authormes = Message.split(': ') # si existe el autor y Message no esta vacio, dividir Message en el signo :
        Author =authormes[0]            # almacena la primera parte de Message que correponde al autor
        Message = ' '.join(authormes[1:])   # almacena la segunda parte de Message en Message
    else:
        Author = None
    return Date, Time, Format, Author, Message

## Función que analiza el archivo completo y crea un DataFrame con los datos

In [5]:

def DataFrame_Data(FilePath):

    parsedData = []
    # ruta del archivo

    # abrir archivo
    with open(FilePath, encoding="utf-8") as fp:
        
        messageBuffer = []
        Date, Time, Format, Author = None, None, None, None

        # Recorremos el archivo linea por linea 
        while True:
            # abrir linea por linea
            line = fp.readline()   

            # si no existe la linea, salir del archivo
            if not line:
                break               
        
            # elimina espacios vacios al principio y al final de la linea
            line = line.strip()     
       
            # si hay una fecha en la linea y si no es vacia, añade a pasedData las variables
            if Date_Chat(line):          
                if len(line) > 0:   
                    parsedData.append([Date, Time, Format, Author, ' '.join(messageBuffer)])  
            
                # limpia la lista que almacena el mensaje
                messageBuffer.clear()   

                # añade a cada variable su correpondiente valor de linea
                Date, Time, Format, Author, Message = DataPoint(line)   
            
                # añade el mensaje a la lista messageBuffer
                messageBuffer.append(Message)   
            else:
                messageBuffer.append(line)

    # crear dataframe con los datos tratados
    df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Format', 'Author', 'Message'])
    return df


## DataFrame con los datos seccionados

Pasamos la ruta donde se encuentra el archivo con los datos de Whatsapp y la asignamos a una variable que representara el DataFrame

In [6]:
ruta_archivo = '../data/chat_raw.txt'

df = DataFrame_Data(ruta_archivo)
df

,Date,Time,Format,Author,Message
0,None,None,None,None,
1,20/11/21,2:30,p. m.,Los mensajes y las llamadas están cifrados de ...,
2,26/5/19,10:48,p. m.,"El creador del grupo creó el grupo ""Comercio d...",
3,20/11/21,2:30,p. m.,Te uniste usando el enlace de invitación de es...,
4,20/11/21,3:37,p. m.,+57 304 4051275,<Multimedia omitido>
...,...,...,...,...,...
8354,29/4/22,5:11,p. m.,+57 304 4051275,<Multimedia omitido>
8355,29/4/22,6:32,p. m.,‎+57 321 8884816 salió del grupo,
8356,29/4/22,7:50,p. m.,+57 304 4051275,<Multimedia omitido>
8357,29/4/22,8:03,p. m.,+57 300 8710844,<Multimedia omitido>


# Limpieza de los Datos

Una vez pasada la información al formato de DataFrame, continuamos limpiando los datos de manera que sólo nos queden información relevante para ser analizada.

## Eliminar registros no deseados

El primer registro no proporciona información de valor por lo que lo eliminamos.

In [7]:
df = df.drop(range(0,1))

# reseteamos los indices
df = df.reset_index(drop=True)
df

,Date,Time,Format,Author,Message
0,20/11/21,2:30,p. m.,Los mensajes y las llamadas están cifrados de ...,
1,26/5/19,10:48,p. m.,"El creador del grupo creó el grupo ""Comercio d...",
2,20/11/21,2:30,p. m.,Te uniste usando el enlace de invitación de es...,
3,20/11/21,3:37,p. m.,+57 304 4051275,<Multimedia omitido>
4,20/11/21,3:53,p. m.,+57 304 6275684,<Multimedia omitido>
...,...,...,...,...,...
8353,29/4/22,5:11,p. m.,+57 304 4051275,<Multimedia omitido>
8354,29/4/22,6:32,p. m.,‎+57 321 8884816 salió del grupo,
8355,29/4/22,7:50,p. m.,+57 304 4051275,<Multimedia omitido>
8356,29/4/22,8:03,p. m.,+57 300 8710844,<Multimedia omitido>


## Eliminar registros con mensajes vacios

In [8]:
df = df.drop(df[df['Message'] == ''].index)

# reseteamos los indices para los registros restantes
df = df.reset_index(drop=True)

df

,Date,Time,Format,Author,Message
0,20/11/21,3:37,p. m.,+57 304 4051275,<Multimedia omitido>
1,20/11/21,3:53,p. m.,+57 304 6275684,<Multimedia omitido>
2,20/11/21,3:57,p. m.,+57 313 7691044,<Multimedia omitido>
3,20/11/21,4:20,p. m.,+57 319 4926821,Buenas tardes!! Tienen el número de la carpint...
4,20/11/21,4:26,p. m.,Amor Bello,<Multimedia omitido>
...,...,...,...,...,...
8257,29/4/22,3:57,p. m.,+57 301 6518988,<Multimedia omitido>
8258,29/4/22,5:11,p. m.,+57 304 4051275,<Multimedia omitido>
8259,29/4/22,7:50,p. m.,+57 304 4051275,<Multimedia omitido>
8260,29/4/22,8:03,p. m.,+57 300 8710844,<Multimedia omitido>


## Cantidad de miembros en el grupo

In [9]:
len(df['Author'].unique())

233

## Proteger información de los miembros del grupo

Realizamos la protección de los telefonos debido a que es información sensible. Para ello, creamos una lista con los miembros del grupo.

In [19]:
miembros = df['Author'].unique()
miembros.shape

(233,)

Cambiamos los teléfonos y nombres de contacto por unos mas convencionales.

In [11]:
for i in enumerate(miembros):
    df['Author'] = df['Author'].replace(i[1], f'user {i[0]}')

df

,Date,Time,Format,Author,Message
0,20/11/21,3:37,p. m.,user 0,<Multimedia omitido>
1,20/11/21,3:53,p. m.,user 1,<Multimedia omitido>
2,20/11/21,3:57,p. m.,user 2,<Multimedia omitido>
3,20/11/21,4:20,p. m.,user 3,Buenas tardes!! Tienen el número de la carpint...
4,20/11/21,4:26,p. m.,user 4,<Multimedia omitido>
...,...,...,...,...,...
8257,29/4/22,3:57,p. m.,user 28,<Multimedia omitido>
8258,29/4/22,5:11,p. m.,user 0,<Multimedia omitido>
8259,29/4/22,7:50,p. m.,user 0,<Multimedia omitido>
8260,29/4/22,8:03,p. m.,user 14,<Multimedia omitido>


## Proteger información sensible en los Mensajes

Vamos a reemplazar los teléfonos, los enlaces compartidos y nombres de usuarios contenido en los mensajes para proteger la información.

La siguiente funcion codifica toda informacion sensible contenida en los mensajes para protegerla

In [12]:
def protec_info(string):
    '''
    Funcion que codifica la iinfromacion sensible en los mensajes
    '''

    # codificar numeros de telefonos
    patron = r'\b\d{3}[-\s]?\d{7}\b|\(\d{3}\)[-.\s]?\d{3}[-.\s]?\d{4}\b'
    string = re.sub(patron,' NNNN ',string)

    # codificar links
    patron = r'\bhttps?://\S+\b'
    string = re.sub(patron,'LLLL',string)

    # codificar nombres de usuarios
    patron = r'@\w+\s?'
    string = re.sub(patron,'UUUU ',string)

    # codificar enlaces a whatsapp
    patron = r'\bwa.me/\d+\b'
    string = re.sub(patron,'NNNN',string)
    
    return string

Aplicamos la función anterior a los Message para asi reemplazar los teléfonos, enlaces y nombres de usuarios por palabras de referencia como NNNN, LLLL y UUUU.

In [13]:
df['Message_Private'] = df['Message'].apply(protec_info)

# Eliminamos la columna Message y renombramos la nueva columna
df = df.drop('Message', axis=1)
df = df.rename(columns={'Message_Private':'Message'})

df

,Date,Time,Format,Author,Message
0,20/11/21,3:37,p. m.,user 0,<Multimedia omitido>
1,20/11/21,3:53,p. m.,user 1,<Multimedia omitido>
2,20/11/21,3:57,p. m.,user 2,<Multimedia omitido>
3,20/11/21,4:20,p. m.,user 3,Buenas tardes!! Tienen el número de la carpint...
4,20/11/21,4:26,p. m.,user 4,<Multimedia omitido>
...,...,...,...,...,...
8257,29/4/22,3:57,p. m.,user 28,<Multimedia omitido>
8258,29/4/22,5:11,p. m.,user 0,<Multimedia omitido>
8259,29/4/22,7:50,p. m.,user 0,<Multimedia omitido>
8260,29/4/22,8:03,p. m.,user 14,<Multimedia omitido>


## Eliminar registros que no tienen Autor

Buscamos los registros que no poseen usuario

In [14]:
NoneValue = df[df['Author'].isnull()]

# eliminamos los registros vacios
df = df.drop(NoneValue.index)

## Cambiar formato de hora

In [15]:
# unimos los valores de Format con Time en la columna Time
df['Time'] = df.Time.str.cat(df.Format, sep=' ')

# convertir valores de Time en objetos DateTime 
df['Time'] = pd.to_datetime(df['Time'])

# convertir valores de Time en str
df['Time'] = df['Time'].astype(str)

# Extreaer solo la hora de la columna Time
df['Time'] = df['Time'].str.extract('(..:..:..)', expand=True)

# eliminar columna Format
df = df.drop(['Format'],axis=1)

df


,Date,Time,Author,Message
0,20/11/21,15:37:00,user 0,<Multimedia omitido>
1,20/11/21,15:53:00,user 1,<Multimedia omitido>
2,20/11/21,15:57:00,user 2,<Multimedia omitido>
3,20/11/21,16:20:00,user 3,Buenas tardes!! Tienen el número de la carpint...
4,20/11/21,16:26:00,user 4,<Multimedia omitido>
...,...,...,...,...
8257,29/4/22,15:57:00,user 28,<Multimedia omitido>
8258,29/4/22,17:11:00,user 0,<Multimedia omitido>
8259,29/4/22,19:50:00,user 0,<Multimedia omitido>
8260,29/4/22,20:03:00,user 14,<Multimedia omitido>


## Crear columnas con detalles de fecha

In [16]:
# diccionario con el valor y nombre de los dias
week = {
    6:'Domingo',
    0:'Lunes',
    1:'Martes',
    2:'Miercoles',
    3:'Jueves',
    4:'Viernes',
    5:'Sabado'
}

# Diccionario con el valor y nombre de los meses
month = {
    1:'Ene',
    2:'Feb',
    3:'Mar',
    4:'Abr',
    5:'May',
    6:'Jun',
    7:'Jul',
    8:'Ago',
    9:'Sept',
    10:'Obt',
    11:'Nov',
    12:'Dic'
}


# dar formato de fecha a los dias 
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

# nueva columna con los nombres de los dias
df['Day'] = df['Date'].dt.weekday.map(week)

# nueva columna con el numero del dia
df['Num_Day'] = df['Date'].dt.day

# nueva columna con el año 
df['Year'] = df['Date'].dt.year

# nueva columna con el numero de mes
df['Num_Month'] = df['Date'].dt.month

# nueva columna con el nombre de los meses
df['Month'] = df['Date'].dt.month.map(month)

# organizar las columnas
df = df[['Date', 'Day', 'Num_Day','Num_Month', 'Month', 'Year', 'Time', 'Author', 'Message']]

# cambiar el tipo de dato de la columna Day
df['Day'] = df['Day'].astype('category')

df

,Date,Day,Num_Day,Num_Month,Month,Year,Time,Author,Message
0,2021-11-20,Sabado,20,11,Nov,2021,15:37:00,user 0,<Multimedia omitido>
1,2021-11-20,Sabado,20,11,Nov,2021,15:53:00,user 1,<Multimedia omitido>
2,2021-11-20,Sabado,20,11,Nov,2021,15:57:00,user 2,<Multimedia omitido>
3,2021-11-20,Sabado,20,11,Nov,2021,16:20:00,user 3,Buenas tardes!! Tienen el número de la carpint...
4,2021-11-20,Sabado,20,11,Nov,2021,16:26:00,user 4,<Multimedia omitido>
...,...,...,...,...,...,...,...,...,...
8257,2022-04-29,Viernes,29,4,Abr,2022,15:57:00,user 28,<Multimedia omitido>
8258,2022-04-29,Viernes,29,4,Abr,2022,17:11:00,user 0,<Multimedia omitido>
8259,2022-04-29,Viernes,29,4,Abr,2022,19:50:00,user 0,<Multimedia omitido>
8260,2022-04-29,Viernes,29,4,Abr,2022,20:03:00,user 14,<Multimedia omitido>


## Cambiar el formato de fecha

In [17]:
df['fecha'] = pd.to_datetime(df['Date'])

# nueva columna con el formato indicado
df['fecha'] = df['fecha'].dt.strftime('%d/%m/%Y')

# eliminar columna Date 
df = df.drop(['Date'],axis=1)

# renombrar columna fecha como Date
df = df.rename(columns={'fecha':'Date'})

# ordenar columnas del dataframe
df = df[['Date','Day', 'Num_Day', 'Month', 'Num_Month' , 'Year', 'Time', 'Author', 'Message']]
df

,Date,Day,Num_Day,Month,Num_Month,Year,Time,Author,Message
0,20/11/2021,Sabado,20,Nov,11,2021,15:37:00,user 0,<Multimedia omitido>
1,20/11/2021,Sabado,20,Nov,11,2021,15:53:00,user 1,<Multimedia omitido>
2,20/11/2021,Sabado,20,Nov,11,2021,15:57:00,user 2,<Multimedia omitido>
3,20/11/2021,Sabado,20,Nov,11,2021,16:20:00,user 3,Buenas tardes!! Tienen el número de la carpint...
4,20/11/2021,Sabado,20,Nov,11,2021,16:26:00,user 4,<Multimedia omitido>
...,...,...,...,...,...,...,...,...,...
8257,29/04/2022,Viernes,29,Abr,4,2022,15:57:00,user 28,<Multimedia omitido>
8258,29/04/2022,Viernes,29,Abr,4,2022,17:11:00,user 0,<Multimedia omitido>
8259,29/04/2022,Viernes,29,Abr,4,2022,19:50:00,user 0,<Multimedia omitido>
8260,29/04/2022,Viernes,29,Abr,4,2022,20:03:00,user 14,<Multimedia omitido>


# Exportación de los Datos Preparados

In [18]:
# Exportamos los datos limpios en un archivo csv
df.to_csv('../data/data_clear.csv', header=True, index=False)